In [ ]:
import numpy as np 
import pandas as pd 
import os
import os
import zipfile
import pandas as pd
import numpy as np
import csv
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import shutil

In [ ]:
a=[]
b=[]
with open('../input/retinal-disease-classification/Training_Set/Training_Set/RFMiD_Training_Labels.csv','r') as F:
  reader=csv.reader(F)
  for row in reader:
      if(row[1]=='1'):
        a.append(row[0]+'.png')
      if(row[1]=='0'):
        b.append(row[0]+'.png')
F.close() 
os.mkdir('./Training')

In [ ]:
parent_dir="./Training"
directory1 = "Abnormal" 
directory2 = "Normal"
path1 = os.path.join(parent_dir, directory1)
path2 = os.path.join(parent_dir, directory2)
os.mkdir(path1)
os.mkdir(path2)

In [ ]:
save_path = './Training/Abnormal'
for filename in a:
  shutil.copy('../input/retinal-disease-classification/Training_Set/Training_Set/Training/'+filename,save_path)
save_path = './Training/Normal'
for filename in b:
  shutil.copy('../input/retinal-disease-classification/Training_Set/Training_Set/Training/'+filename,save_path)

In [ ]:
temp=[]
for dirname, _, filenames in os.walk('../input/normaleyes/1_normal'):
    for filename in filenames:
        path=os.path.join(dirname, filename)
        temp.append(path)
save_path = './Training/Normal'
for filename in temp:
    shutil.copy(filename,save_path)

In [ ]:
a=[]
b=[]
with open('../input/retinal-disease-classification/Evaluation_Set/Evaluation_Set/RFMiD_Validation_Labels.csv','r') as F:
  reader=csv.reader(F)
  for row in reader:
      if(row[1]=='1'):
        a.append(row[0]+'.png')
      if(row[1]=='0'):
        b.append(row[0]+'.png')
F.close() 
os.mkdir('./Validation')

In [ ]:
parent_dir="./Validation"
directory1 = "Abnormal" 
directory2 = "Normal"
path1 = os.path.join(parent_dir, directory1)
path2 = os.path.join(parent_dir, directory2)
os.mkdir(path1)
os.mkdir(path2)

In [ ]:
save_path = './Validation/Abnormal'
for filename in a:
  shutil.copy('../input/retinal-disease-classification/Training_Set/Training_Set/Training/'+filename,save_path)
save_path = './Validation/Normal'
for filename in b:
  shutil.copy('../input/retinal-disease-classification/Training_Set/Training_Set/Training/'+filename,save_path)

In [ ]:
train_abnormal_dir = os.path.join('./Training/Abnormal')
train_normal_dir = os.path.join('./Training/Normal')
train_abnormal_names = os.listdir(train_abnormal_dir)
train_normal_names = os.listdir(train_normal_dir)
print('total training images with abnormalities:', len(os.listdir(train_abnormal_dir)))
print('total training images of normal eyes:', len(os.listdir(train_normal_dir)))

In [ ]:
validation_abnormal_dir = os.path.join('./Validation/Abnormal')
validation_normal_dir = os.path.join('./Validation/Normal')
validation_abnormal_names = os.listdir(validation_abnormal_dir)
validation_normal_names = os.listdir(validation_normal_dir)
print('total validation images with abnormalities:', len(os.listdir(validation_abnormal_dir)))
print('total validation images of normal eyes:', len(os.listdir(validation_normal_dir)))

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                    rotation_range = 30,
                                    width_shift_range = 0.1,
                                    height_shift_range = 0.1,
                                    zoom_range = 0.1,
                                    horizontal_flip = True
                                   )
train_generator = train_datagen.flow_from_directory('./Training', 
                                                    target_size=(150,150), 
                                                    class_mode='binary')

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1/255)
validation_generator = validation_datagen.flow_from_directory('./Validation', 
                                                              target_size=(150,150),
                                                              class_mode='binary')

In [ ]:
local_weights_file = '../input/inceptionv3weights/inception_v3_weights_tf_dim_ordering_tf_kernels_notop (1).h5'
pre_trained_model = InceptionV3(input_shape = (150,150, 3), 
                                include_top = False, 
                                weights = None)
pre_trained_model.load_weights(local_weights_file)
for layer in pre_trained_model.layers:
  layer.trainable = False
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

In [ ]:
x = layers.Flatten()(last_output)
x = layers.Dropout(0.5)(x) 
x = layers.Dense(1024, activation='relu')(x)                 
x = layers.Dense (1, activation='sigmoid')(x)           
model = Model( pre_trained_model.input, x) 
model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
device_name = tf.test.gpu_device_name()
if "GPU" not in device_name:
    print("GPU device not found")
print('Found GPU at: {}'.format(device_name))
history = model.fit(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch=20,
            epochs =25,
            validation_steps=5,
            verbose = 1)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()